In [2]:
import numpy as np
import datetime
import random
import sympy as sym
from sympy import *
from scipy.optimize import minimize
import math
from sympy import symbols
from sympy.plotting import plot, plot3d, PlotGrid

In [3]:
def Pr(xt_,yt,c,alpha,beta,gama,theta_0,theta_1):
        
    if yt==0:
        p0=1-theta_0
        p1=1-theta_1
    else:
        p0=theta_0
        p1=theta_1
    if xt_==0:
        p0=p0*(1-alpha)*(1-beta)**c
        p1=p1*(1-(1-alpha)*(1-beta)**c)
    else:
        p0=p0*gama
        p1=p1*(1-gama)

      
    return p0,p1
                

In [4]:
# Function to calculate emission probability:
def emission(X_,Y_,n,T,q_):
    theta_0 = Symbol('theta_0')
    theta_1 = Symbol('theta_1')
    P=[]
    for j in range(q_):
        p=1
        for m in range(n):
            for t in range(T):
                if X_[j][m][t]==0:
                    if Y_[j][m][t]==1:
                        p=p*theta_0
                    else:
                        p=p*(1-theta_0)
                else:  
                    if Y_[j][m][t]==1:
                        p=p*theta_1
                    else:
                        p=p*(1-theta_1)
        P.append(sym.log(p))                 
    return P                

In [5]:
# Function to calculate transmition probability:
def transition(X_,C_,n,T,q_):
    alpha = Symbol('alpha')
    beta= Symbol('beta')
    gamma=Symbol('gamma')
    pi=Symbol('pi')
    P=[]
    for j in range(q_):
        p=1
        for m in range(n):
            for t in range(T-1):
                if (X_[j][m][t]==0):
                    if t==0:
                        p=p*pi
                    if (X_[j][m][t+1]==0):
                        p=p*(1-alpha)*(1-beta)**C_[j][m][t]
                    else:
                        p=p*(1-(1-alpha)*(1-beta)**C_[j][m][t])
                else: 
                    if t==0:
                        p=p*(1-pi)
                    if (X_[j][m][t+1]==0):
                        p=p*gamma
                    else:
                        p=p*(1-gamma)
        P.append(sym.log(p))
        
    return P         
    

In [6]:
def CNbr(G_,X_):
    C_=[]
    for k in range(len(np.array(X_))):
        indx=0
        X=X_[k] 
        G=G_[k]
        C=np.zeros((G.shape[1],G.shape[0]))
        for t in range(G.shape[0]):
            for i in range(G.shape[1]):
                indx=0
                c=0
                g= G[t,i]
                for k in g:
                    if k==1:
                        if X[indx,t]==1:
                            c=c+1
                    indx=indx+1
                C[i,t]=c
        C_.append(C)        
    return C_      

In [7]:
alpha = Symbol('alpha')
beta= Symbol('beta')
gamma=Symbol('gamma')
theta_0 = Symbol('theta_0')
theta_1 = Symbol('theta_1')

In [22]:
#******************************update parameters
alpha_=0.5
beta_=0.5
gama_=0.5
theta_0_=0.5
theta_1_=0.5
p_=0.8
n=100
T=100
J=20
q_=3
X=np.zeros((n,T))

In [23]:
def Calc(q_,J,T,alpha_,beta_,gama_,theta_0_,theta_1_):
    X_=[]
    G_=[]
    Y_=[]
    
    for q in range(q_):
        X=np.zeros((n,T))
        G=[]
        for j in range(T-1):
            g=np.identity(100)
            for i in range(100):
                inx=np.random.choice(range(i,100), 10)
                g[i,inx]=1  
                g[inx,i]=1
            G.append(g)
        G=np.array(G)
        G_.append(G)
        a=[]
        Y=np.zeros((n,T))
        idx=np.random.choice(range(n), int((1-p_)*n))
        Y[idx,0]=1
        for t in range(T-1):
            for i in range(n):
                for j in range(n):
                    if G[t][i][j]==1:
                        a.append((i,j))
                        if Y[i,t]==1:
                            if Y[j,t]==1:
                                Y[i,t+1]=Y[j,t+1]=0
                        else:
                            Y[i,t+1]=0
                            Y[j,t+1]=1
                    elif Y[j,t]==1:
                        Y[i,t+1]=1
                        Y[j,t+1]=0
        Y_.append(Y)
        for j in range(J):    
            for t in range(1,T):
                c_=G[t-1].dot(X).T[t-1]
                for i in range(n):
            
                    p0=Pr(X[i,t-1],Y[i,t],c_[i],alpha_,beta_,gama_,theta_0_,theta_1_)[0]
                    p1=Pr(X[i,t-1],Y[i,t],c_[i],alpha_,beta_,gama_,theta_0_,theta_1_)[1]
                    if t!=T-1:
                        c=G[t].dot(X).T[t]
                    
                        g=G[t,i]
                        for k in g:
                            k=int(k)
                            if X[k][t]==0:
                                if X[k][t+1]==0:
                                    if X[i,t]==0:
                                        p0=p0*(1-alpha_)*(1-beta_)**c[k]
                                    else:
                                        p1=p1*(1-alpha_)*(1-beta_)**c[k]
                                else:
                                    if X[i,t]==0:
                                        p0=p0*(1-(1-alpha_)*(1-beta_)**c[k])
                                    else:
                                        p1=p1*(1-(1-alpha_)*(1-beta_)**c[k])
                            elif X[k][t+1]==0:
                                if X[i,t]==0:
                                    p0=p0*gama_
                                else:
                                    p1=p1*gama_
                            elif X[i,t]==0:
                                p0=p0*(1-gama_)
                            else:
                                p1=p1*(1-gama_)
                    if p0>p1:
                        X[i,t]=0
                    else:
                        X[i,t]=1  
            if j!=J:
                for i in range(n):
                    indx=0
                    p0=0
                    p1=0
                    c=0
                    g= G[0,i]
                    if Y[i,0]==0:
                        p1=1-theta_1_
                        p0=1-theta_0_
                    else:
                        p1=theta_1_
                        p0=theta_0_
                    p0=p0*p_
                    p1=p1*(1-p_)
                    for k in g:
                        if k==1:
                            if X[indx,1]==1:
                                c=c+1
                        indx=indx+1
                    indx=0 
                    for k in g:
                        if X[indx,1]==1:
                            if X[indx,0]==0:
                                if indx==i:
                                    c=c-1
                                p0=p0*(1-alpha_)*(1-beta_)**c
                                p1=p1*(1-alpha_)*(1-beta_)**c
                            else:
                                if indx==i:
                                    c=c-1
                                p0=p0*(1-gama_)
                                p1=p1*(1-gama_)
                        else :
                            if X[indx,0]==0:
                                if indx==i:
                                    c=c-1
                                p0=p0*(1-(1-alpha_)*(1-beta_)**c)
                                p1=p1*(1-(1-alpha_)*(1-beta_)**c)
                            else:
                                if indx==i:
                                    c=c-1
                                p1=p1*gama_
                                p0=p0*gama_
                    if p0>p1:
                        X[i,0]=0
                    else: 
                        X[i,0]=1
        X_.append(X)
    G_=np.array(G_)
    X_=np.array(X_)
    Y_=np.array(Y_)
    return X_,Y_,G_    

In [24]:
U=Calc(q_,J,T,alpha_,beta_,gama_,theta_0_,theta_1_)
X_=U[0]
Y_=U[1]
G_=U[2]

In [25]:
Q=np.sum(np.array(transition(X_,CNbr(G_,X_),n,T,q_))+emission(X_,Y_,n,T,q_),axis=0)
Q

log(theta_0**1970*theta_1**3014*(1 - theta_0)**2772*(1 - theta_1)**2244) + log(theta_0**2390*theta_1**2594*(1 - theta_0)**2394*(1 - theta_1)**2622) + log(theta_0**2477*theta_1**2536*(1 - theta_0)**2221*(1 - theta_1)**2766) + log(2.54629497041812e-10*alpha**94*gamma**4680*(0.2 - 0.2*gamma)*(1 - alpha)**5*(1 - beta)**5.0*(1 - gamma)**435*(-(1 - alpha)*(1 - beta)**1.0 + 1)**42*(-(1 - alpha)*(1 - beta)**2.0 + 1)**151*(-(1 - alpha)*(1 - beta)**3.0 + 1)**256*(-(1 - alpha)*(1 - beta)**4.0 + 1)**431*(-(1 - alpha)*(1 - beta)**5.0 + 1)**522*(-(1 - alpha)*(1 - beta)**6.0 + 1)**612*(-(1 - alpha)*(1 - beta)**7.0 + 1)**542*(-(1 - alpha)*(1 - beta)**8.0 + 1)**507*(-(1 - alpha)*(1 - beta)**9.0 + 1)**402*(-(1 - alpha)*(1 - beta)**10.0 + 1)**306*(-(1 - alpha)*(1 - beta)**11.0 + 1)**245*(-(1 - alpha)*(1 - beta)**12.0 + 1)**194*(-(1 - alpha)*(1 - beta)**13.0 + 1)**139*(-(1 - alpha)*(1 - beta)**14.0 + 1)**117*(-(1 - alpha)*(1 - beta)**15.0 + 1)**84*(-(1 - alpha)*(1 - beta)**16.0 + 1)**57*(-(1 - alpha)*(1 -

In [26]:
simplify(factor(Q.diff(gamma)))

(15476.0*gamma - 13915.0)/(gamma*(gamma - 1))

In [27]:
a=float(solve(simplify(Q.diff(theta_0)))[0])
b=float(solve(simplify(Q.diff(theta_1)))[0])
ga=float(solve(simplify(factor(Q.diff(gamma))))[0])
a,b,ga

(0.48066647919010125, 0.5162271805273834, 0.8991341431894546)

In [28]:
o=Q.subs(theta_0,a).subs(theta_1,b).subs(gamma,ga)
o

log(1.72987009626388e-661*alpha**94*(1 - alpha)**5*(1 - beta)**5.0*(-(1 - alpha)*(1 - beta)**1.0 + 1)**42*(-(1 - alpha)*(1 - beta)**2.0 + 1)**151*(-(1 - alpha)*(1 - beta)**3.0 + 1)**256*(-(1 - alpha)*(1 - beta)**4.0 + 1)**431*(-(1 - alpha)*(1 - beta)**5.0 + 1)**522*(-(1 - alpha)*(1 - beta)**6.0 + 1)**612*(-(1 - alpha)*(1 - beta)**7.0 + 1)**542*(-(1 - alpha)*(1 - beta)**8.0 + 1)**507*(-(1 - alpha)*(1 - beta)**9.0 + 1)**402*(-(1 - alpha)*(1 - beta)**10.0 + 1)**306*(-(1 - alpha)*(1 - beta)**11.0 + 1)**245*(-(1 - alpha)*(1 - beta)**12.0 + 1)**194*(-(1 - alpha)*(1 - beta)**13.0 + 1)**139*(-(1 - alpha)*(1 - beta)**14.0 + 1)**117*(-(1 - alpha)*(1 - beta)**15.0 + 1)**84*(-(1 - alpha)*(1 - beta)**16.0 + 1)**57*(-(1 - alpha)*(1 - beta)**17.0 + 1)**33*(-(1 - alpha)*(1 - beta)**18.0 + 1)**21*(-(1 - alpha)*(1 - beta)**19.0 + 1)**10*(-(1 - alpha)*(1 - beta)**20.0 + 1)**4*(-(1 - alpha)*(1 - beta)**21.0 + 1)**6*(-(1 - alpha)*(1 - beta)**22.0 + 1)**3*(-(1 - alpha)*(1 - beta)**26.0 + 1)) + log(2.4663620

In [29]:
o=expand_log(o, force=True)
#e=o.subs(alpha,.2765)
o

282*log(alpha) + 12*log(1 - alpha) + 12.0*log(1 - beta) + 179*log(-(1 - alpha)*(1 - beta)**1.0 + 1) + 523*log(-(1 - alpha)*(1 - beta)**2.0 + 1) + 922*log(-(1 - alpha)*(1 - beta)**3.0 + 1) + 1339*log(-(1 - alpha)*(1 - beta)**4.0 + 1) + 1660*log(-(1 - alpha)*(1 - beta)**5.0 + 1) + 1748*log(-(1 - alpha)*(1 - beta)**6.0 + 1) + 1617*log(-(1 - alpha)*(1 - beta)**7.0 + 1) + 1419*log(-(1 - alpha)*(1 - beta)**8.0 + 1) + 1160*log(-(1 - alpha)*(1 - beta)**9.0 + 1) + 883*log(-(1 - alpha)*(1 - beta)**10.0 + 1) + 667*log(-(1 - alpha)*(1 - beta)**11.0 + 1) + 555*log(-(1 - alpha)*(1 - beta)**12.0 + 1) + 388*log(-(1 - alpha)*(1 - beta)**13.0 + 1) + 285*log(-(1 - alpha)*(1 - beta)**14.0 + 1) + 211*log(-(1 - alpha)*(1 - beta)**15.0 + 1) + 141*log(-(1 - alpha)*(1 - beta)**16.0 + 1) + 93*log(-(1 - alpha)*(1 - beta)**17.0 + 1) + 52*log(-(1 - alpha)*(1 - beta)**18.0 + 1) + 41*log(-(1 - alpha)*(1 - beta)**19.0 + 1) + 18*log(-(1 - alpha)*(1 - beta)**20.0 + 1) + 14*log(-(1 - alpha)*(1 - beta)**21.0 + 1) + 5*log

In [30]:
alta=nsolve((o.diff(alpha),o.diff(beta)),(alpha,beta),(0.1496,0.4))

In [31]:
#******************************update parameters
alpha_=alta[0]
beta_=alta[1]
gama_=ga
theta_0_=a
theta_1_=b

In [38]:
print("𝛼:",alpha_,"𝛽:",beta_,"𝛾:",gama_,"𝜃_0:",theta_0_,"𝜃_1",theta_1_,"\n")

𝛼: 0.976067252877729 𝛽: 0.641237016174635 𝛾: 0.8991341431894546 𝜃_0: 0.48066647919010125 𝜃_1 0.5162271805273834 



In [32]:
def CalcX(Y_,G_,q_,J,T,alpha_,beta_,gama_,theta_0_,theta_1_):
    X_=[]
    p0=0
    p1=0
    
    for q in range(q_):
        #X=np.zeros((n,T))
        G=G_[q]
        Y=Y_[q]
        for j in range(J):    
            for t in range(1,T):
                c_=G[t-1].dot(X).T[t-1]
                for i in range(n):
            
                    p0=Pr(X[i,t-1],Y[i,t],c_[i],alpha_,beta_,gama_,theta_0_,theta_1_)[0]
                    p1=Pr(X[i,t-1],Y[i,t],c_[i],alpha_,beta_,gama_,theta_0_,theta_1_)[1]
                    if t!=T-1:
                        c=G[t].dot(X).T[t]
                    
                        g=G[t,i]
                        for k in g:
                            k=int(k)
                            if X[k][t]==0:
                                if X[k][t+1]==0:
                                    if X[i,t]==0:
                                        p0=p0*(1-alpha_)*(1-beta_)**c[k]
                                    else:
                                        p1=p1*(1-alpha_)*(1-beta_)**c[k]
                                else:
                                    if X[i,t]==0:
                                        p0=p0*(1-(1-alpha_)*(1-beta_)**c[k])
                                    else:
                                        p1=p1*(1-(1-alpha_)*(1-beta_)**c[k])
                            elif X[k][t+1]==0:
                                if X[i,t]==0:
                                    p0=p0*gama_
                                else:
                                    p1=p1*gama_
                            elif X[i,t]==0:
                                p0=p0*(1-gama_)
                            else:
                                p1=p1*(1-gama_)
                    if p0>p1:
                        X[i,t]=0
                    else:
                        X[i,t]=1  
            if j!=J:
                for i in range(n):
                    indx=0
                    p0=0
                    p1=0
                    c=0
                    g= G[0,i]
                    if Y[i,0]==0:
                        p1=1-theta_1_
                        p0=1-theta_0_
                    else:
                        p1=theta_1_
                        p0=theta_0_
                    p0=p0*p_
                    p1=p1*(1-p_)
                    for k in g:
                        if k==1:
                            if X[indx,1]==1:
                                c=c+1
                        indx=indx+1
                    indx=0 
                    for k in g:
                        if X[indx,1]==1:
                            if X[indx,0]==0:
                                if indx==i:
                                    c=c-1
                                p0=p0*(1-alpha_)*(1-beta_)**c
                                p1=p1*(1-alpha_)*(1-beta_)**c
                            else:
                                if indx==i:
                                    c=c-1
                                p0=p0*(1-gama_)
                                p1=p1*(1-gama_)
                        else :
                            if X[indx,0]==0:
                                if indx==i:
                                    c=c-1
                                p0=p0*(1-(1-alpha_)*(1-beta_)**c)
                                p1=p1*(1-(1-alpha_)*(1-beta_)**c)
                            else:
                                if indx==i:
                                    c=c-1
                                p1=p1*gama_
                                p0=p0*gama_
                    if p0>p1:
                        X[i,0]=0
                    else: 
                        X[i,0]=1
        X_.append(X)
    
    return X_   

In [36]:
J=20

In [37]:
print("Parameter initialization:................................................................")
print("𝛼:",alpha_,"  𝛽:",beta_,"  𝛾:",gama_,"  𝜃_0:",theta_0_,"  𝜃_1",theta_1_)
print("Updating X:.............................................................................\n",X)
for i in range(2):
    print("Current time:",datetime.datetime.now())
    print("Iterration",i+1,":.........................................................................")
    #X=np.zeros((n,T))
    print("Computing X:...........................................................................")
    X_=CalcX(Y_,G_,q_,J,T,alpha_,beta_,gama_,theta_0_,theta_1_)
    print("Current time:",datetime.datetime.now())
    print("X:\n",X,"\n")
    print("Computing Q:..........................................................................")
    Q=np.sum(np.array(transition(X_,CNbr(G_,X_),n,T,q_))+emission(X_,Y_,n,T,q_),axis=0)
    print("Current time:",datetime.datetime.now())
    print("Q:\n",Q)
    if not solve(simplify(factor(Q.diff(gamma)))):
        print("error")
        break
    ga=float(solve(simplify(factor(Q.diff(gamma))))[0])
    a=float(solve(simplify(Q.diff(theta_0)))[0])
    b=float(solve(simplify(Q.diff(theta_1)))[0]) 
    o=Q.subs(theta_0,a).subs(theta_1,b).subs(gamma,ga)
    print("o:\n",o)
    o=expand_log(o, force=True)
    print("o_:\n",o)
    #plot3d(o)
    alta=nsolve((o.diff(beta),o.diff(alpha)),(beta,alpha),(.05,.5))
    alpha_=alta[1]
    beta_=alta[0]
    gama_=c
    theta_0_=a
    theta_1_=b
    print("Updating parameters:.............................................................................\n")
    print("𝛼:",alpha_,"𝛽:",beta_,"𝛾:",gama_,"𝜃_0:",theta_0_,"𝜃_1",theta_1_,"\n")
    print("Updating X:.............................................................................\n",X,"\n")

Parameter initialization:................................................................
𝛼: 0.976067252877729   𝛽: 0.641237016174635   𝛾: 0.8991341431894546   𝜃_0: 0.48066647919010125   𝜃_1 0.5162271805273834
Updating X:.............................................................................
 [[1. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 1. 0. 1.]
 ...
 [0. 1. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 1. 0. 1.]]
Current time: 2021-10-27 10:15:01.527813
Iterration 1 :.........................................................................
Computing X:...........................................................................
Current time: 2021-10-27 11:18:29.044915
X:
 [[1. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 1. 0. 1.]
 ...
 [0. 1. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 1. 0. 1.]] 

Computing Q:..........................................................................
Current time: 2021-10-27 11:19:57.807

ValueError: Could not find root within given tolerance. (1845.95835737971661567 > 2.16840434497100886801e-19)
Try another starting point or tweak arguments.